In [18]:
from pyspark.sql import SparkSession, functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [ ]:
spark: SparkSession = SparkSession.builder.appName("PopularMovies").getOrCreate()

24/11/25 20:53:14 WARN Utils: Your hostname, Ngas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.115 instead (on interface en0)
24/11/25 20:53:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/25 20:53:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/25 20:53:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [19]:
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("movie_id", IntegerType(), True),
    StructField("rating", DoubleType(), True),
    StructField("timestamp", IntegerType(), True)
])

In [22]:
df = spark.read.option("header", "true").schema(schema).csv("../data/ml-32m/ratings.csv")
df.show()


+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|      1|      17|   4.0|944249077|
|      1|      25|   1.0|944250228|
|      1|      29|   2.0|943230976|
|      1|      30|   5.0|944249077|
|      1|      32|   5.0|943228858|
|      1|      34|   2.0|943228491|
|      1|      36|   1.0|944249008|
|      1|      80|   5.0|944248943|
|      1|     110|   3.0|943231119|
|      1|     111|   5.0|944249008|
|      1|     161|   1.0|943231162|
|      1|     166|   5.0|943228442|
|      1|     176|   4.0|944079496|
|      1|     223|   3.0|944082810|
|      1|     232|   5.0|943228442|
|      1|     260|   5.0|943228696|
|      1|     302|   4.0|944253272|
|      1|     306|   5.0|944248888|
|      1|     307|   5.0|944253207|
|      1|     322|   4.0|944053801|
+-------+--------+------+---------+
only showing top 20 rows



24/11/25 21:02:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: userId, movieId, rating, timestamp
 Schema: user_id, movie_id, rating, timestamp
Expected: user_id but found: userId
CSV file: file:///Users/ngahoang/Documents/workspace/personal/udemy/spark-with-python/data/ml-32m/ratings.csv


In [26]:
movie_counts = df.groupBy("movie_id").count()
movie_counts.show()

24/11/25 21:14:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: movieId
 Schema: movie_id
Expected: movie_id but found: movieId
CSV file: file:///Users/ngahoang/Documents/workspace/personal/udemy/spark-with-python/data/ml-32m/ratings.csv


+--------+-----+
|movie_id|count|
+--------+-----+
|    3175|17512|
|     471|12016|
|    1580|48434|
|   44022| 6540|
|   68135| 3678|
|  166558|  735|
|    1088|14112|
|    1238| 3342|
|    1342| 4288|
|    1645|16718|
|    1959| 5360|
|    1591| 6188|
|    3794|  849|
|    3997| 2477|
|    4519| 3060|
|    4818|   50|
|    4900|  373|
|    5803| 1473|
|    6620| 4488|
|    7340|  395|
+--------+-----+
only showing top 20 rows



In [24]:
def get_movie_names():
    movie_names = {}
    with open("../data/ml-32m/movies.csv", "r") as f:
        next(f)  # Skip the header line
        for line in f:
            fields = line.split(",")
            movie_id = int(fields[0])
            movie_name = fields[1]
            movie_names[movie_id] = movie_name
    return movie_names

# Broadcast the movie names
movie_names = spark.sparkContext.broadcast(get_movie_names())

# Create a user defined function to look up movie names
def lookup_movie_name(movie_id):
    return movie_names.value.get(movie_id)

lookup_name_udf = func.udf(lookup_movie_name)


In [27]:
movies_with_names = movie_counts.withColumn("movie_name", lookup_name_udf(df["movie_id"]))
movies_with_names.show()

24/11/25 21:14:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: movieId
 Schema: movie_id
Expected: movie_id but found: movieId
CSV file: file:///Users/ngahoang/Documents/workspace/personal/udemy/spark-with-python/data/ml-32m/ratings.csv


+--------+-----+--------------------+
|movie_id|count|          movie_name|
+--------+-----+--------------------+
|    3175|17512| Galaxy Quest (1999)|
|     471|12016|    "Hudsucker Proxy|
|    1580|48434|Men in Black (a.k...|
|   44022| 6540|Ice Age 2: The Me...|
|   68135| 3678|     17 Again (2009)|
|  166558|  735|Underworld: Blood...|
|    1088|14112|Dirty Dancing (1987)|
|    1238| 3342|   Local Hero (1983)|
|    1342| 4288|     Candyman (1992)|
|    1645|16718|The Devil's Advoc...|
|    1959| 5360|Out of Africa (1985)|
|    1591| 6188|        Spawn (1997)|
|    3794|  849| Chuck & Buck (2000)|
|    3997| 2477|Dungeons & Dragon...|
|    4519| 3060|   "Land Before Time|
|    4818|   50| Extreme Days (2001)|
|    4900|  373|     Out Cold (2001)|
|    5803| 1473|        I Spy (2002)|
|    6620| 4488|American Splendor...|
|    7340|  395|Just One of the G...|
+--------+-----+--------------------+
only showing top 20 rows

